In [2]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine
import requests

In [3]:
db_params = {
    "host": "localhost",
    "database": "FoodManagementSystem",
    "user": "postgres",
    "password": "YOUR_PASSWORD"
}

conn = psycopg2.connect(**db_params)
cur = conn.cursor()

In [4]:
# Table names
table_names = [
    "LOCATIONS",
    "RESTAURANTS",
    "RESTAURANTWASTAGE",
    "CUSTOMERS",
    "EMPLOYEES",
    "WAITERS",
    "CHEFS",
    "MENUITEMS",
    "ORDERSTATUSES",
    "ORDERS",
    "ORDERLINE",
    "DELIVERYEMPLOYEES",
    "DELIVERY",
    "DINEINCUSTOMERS",
    "DELIVERYCUSTOMERS",
    "FEEDBACK"
]

In [5]:
# SQL statements to create tables
create_table_statements = {
    "LOCATIONS": """
        CREATE TABLE LOCATIONS (
          location_id INT PRIMARY KEY,
          location_name VARCHAR(50),
          address VARCHAR(100),
          city VARCHAR(50),
          state VARCHAR(50),
          country VARCHAR(50)
        );
    """,
    "RESTAURANTS": """
        CREATE TABLE RESTAURANTS (
          restaurant_id INT PRIMARY KEY,
          restaurant_name VARCHAR(50),
          location_id INT,
          FOREIGN KEY (location_id) REFERENCES LOCATIONS(location_id)
        );
    """,
    "RESTAURANTWASTAGE": """
        CREATE TABLE RESTAURANTWASTAGE (
          wastage_id INT PRIMARY KEY,
          restaurant_id INT,
          date DATE,
          wastage_details VARCHAR(200),
          FOREIGN KEY (restaurant_id) REFERENCES RESTAURANTS(restaurant_id)
        );
    """,
    "CUSTOMERS": """
        CREATE TABLE CUSTOMERS (
          customer_id INT PRIMARY KEY,
          customer_name VARCHAR(50),
          address VARCHAR(100),
          city VARCHAR(50),
          state VARCHAR(50),
          country VARCHAR(50),
          phone_number VARCHAR(20),
          email VARCHAR(50)
        );
    """,
    "EMPLOYEES": """
        CREATE TABLE EMPLOYEES (
          employee_id INT PRIMARY KEY,
          employee_name VARCHAR(50),
          address VARCHAR(100),
          city VARCHAR(50),
          state VARCHAR(50),
          country VARCHAR(50),
          phone_number VARCHAR(20),
          email VARCHAR(50)
        );
    """,
    "WAITERS": """
        CREATE TABLE WAITERS (
          waiter_id INT PRIMARY KEY,
          employee_id INT,
          restaurant_id INT,
          FOREIGN KEY (employee_id) REFERENCES EMPLOYEES(employee_id),
          FOREIGN KEY (restaurant_id) REFERENCES RESTAURANTS(restaurant_id)
        );
    """,
    "CHEFS": """
        CREATE TABLE CHEFS (
          chef_id INT PRIMARY KEY,
          employee_id INT,
          restaurant_id INT,
          FOREIGN KEY (employee_id) REFERENCES EMPLOYEES(employee_id),
          FOREIGN KEY (restaurant_id) REFERENCES RESTAURANTS(restaurant_id)
        );
    """,
    "MENUITEMS": """
        CREATE TABLE MENUITEMS (
          menu_item_id INT PRIMARY KEY,
          restaurant_id INT,
          item_name VARCHAR(50),
          description VARCHAR(200),
          price DECIMAL(8, 2),
          FOREIGN KEY (restaurant_id) REFERENCES RESTAURANTS(restaurant_id)
        );
    """,
    "ORDERSTATUSES": """
        CREATE TABLE ORDERSTATUSES (
          order_status_id INT PRIMARY KEY,
          status_name VARCHAR(50)
        );
    """,
    "ORDERS": """
        CREATE TABLE ORDERS (
```python
          order_id INT PRIMARY KEY,
          customer_id INT,
          waiter_id INT,
          order_status_id INT,
          order_date DATE,
          total_amount DECIMAL(10, 2),
          FOREIGN KEY (customer_id) REFERENCES CUSTOMERS(customer_id),
          FOREIGN KEY (waiter_id) REFERENCES WAITERS(waiter_id),
          FOREIGN KEY (order_status_id) REFERENCES ORDERSTATUSES(order_status_id)
        );
    """,
    "ORDERLINE": """
        CREATE TABLE ORDERLINE (
          orderline_id INT PRIMARY KEY,
          order_id INT,
          menu_item_id INT,
          quantity INT,
          FOREIGN KEY (order_id) REFERENCES ORDERS(order_id),
          FOREIGN KEY (menu_item_id) REFERENCES MENUITEMS(menu_item_id)
        );
    """,
    "DELIVERYEMPLOYEES": """
        CREATE TABLE DELIVERYEMPLOYEES (
          delivery_employee_id INT PRIMARY KEY,
          employee_id INT,
          FOREIGN KEY (employee_id) REFERENCES EMPLOYEES(employee_id)
        );
    """,
    "DELIVERY": """
        CREATE TABLE DELIVERY (
          delivery_id INT PRIMARY KEY,
          order_id INT,
          delivery_employee_id INT,
          delivery_date DATE,
          address VARCHAR(100),
          city VARCHAR(50),
          state VARCHAR(50),
          country VARCHAR(50),
          FOREIGN KEY (order_id) REFERENCES ORDERS(order_id),
          FOREIGN KEY (delivery_employee_id) REFERENCES DELIVERYEMPLOYEES(delivery_employee_id)
        );
    """,
    "DINEINCUSTOMERS": """
        CREATE TABLE DINEINCUSTOMERS (
          customer_id INT PRIMARY KEY,
          table_number INT
        );
    """,
    "DELIVERYCUSTOMERS": """
        CREATE TABLE DELIVERYCUSTOMERS (
          customer_id INT PRIMARY KEY,
          address VARCHAR(100),
          city VARCHAR(50),
          state VARCHAR(50),
          country VARCHAR(50)
        );
    """,
    "FEEDBACK": """
        CREATE TABLE FEEDBACK (
          feedback_id INT PRIMARY KEY,
          customer_id INT,
          feedback_date DATE,
          feedback_text VARCHAR(200),
          FOREIGN KEY (customer_id) REFERENCES CUSTOMERS(customer_id)
        );
    """
}


In [6]:
try:
    # Start a transaction
    conn.autocommit = False

    # Check if tables already exist
    for table_name in table_names:
        cur.execute(
            f"SELECT EXISTS(SELECT 1 FROM information_schema.tables WHERE table_name = '{table_name}')"
        )
        table_exists = cur.fetchone()[0]
        if not table_exists:
            cur.execute(create_table_statements[table_name])

    # Commit the transaction
    conn.commit()

except psycopg2.DatabaseError as e:
    # Rollback the transaction in case of any exception/error
    conn.rollback()
    print("Error:", e)

finally:
    # Restore autocommit to its default value
    conn.autocommit = True

Error: relation "locations" already exists



In [ ]:
# Insert data into LOCATIONS table
cur.execute("""
    INSERT INTO LOCATIONS (location_id, location_name, address, city, state, country)
    VALUES
        (1, 'Kakinada', 'Ashoke nagar', 'Vizag', 'Andhra Pradesh', 'India'),
        (2, 'Sattenapalli', 'Atchempet road', 'Guntur', 'Andhra Pradesh', 'India'),
        (3, 'Eluru', 'Brindhavan gardens', 'Vijayawada', 'Andhra Pradesh', 'India'),
        (4, 'Tirupati', 'Gandhi nagar', 'Chitoor', 'Andhra Pradesh', 'India'),
        (5, 'Gudur', 'Beach road', 'Nellore', 'Andhra Pradesh', 'India');
""")

# Insert data into RESTAURANTS table
cur.execute("""
    INSERT INTO RESTAURANTS (restaurant_id, restaurant_name, location_id)
    VALUES
        (1, 'Sweet magic', 1),
        (2, 'Abhiruchi', 2),
        (3, 'Mourya', 3),
        (4, 'Paradise', 4),
        (5, 'Bawarchi', 5);
""")

# Insert data into RESTAURANTWASTAGE table
cur.execute("""
    INSERT INTO RESTAURANTWASTAGE (wastage_id, restaurant_id, date, wastage_details)
    VALUES
        (1, 1, '2023-07-01', 'Wasted 10kg'),
        (2, 2, '2023-07-02', 'Wasted 17kg'),
        (3, 3, '2023-07-03', 'Wasted 25kg'),
        (4, 4, '2023-07-04', 'Wasted 20kg'),
        (5, 5, '2023-07-05', 'Wasted 15kg');
""")

# Insert data into CUSTOMERS table
cur.execute("""
    INSERT INTO CUSTOMERS (customer_id, customer_name, address, city, state, country, phone_number, email)
    VALUES
        (1, 'Shuban', 'Ashoke nagar', 'Vizag', 'Andhra Pradesh', 'India', '1234567890', 'Shuban@email.com'),
        (2, 'Girish', 'Atchempet road', 'Guntur', 'Andhra Pradesh', 'India', '2345678901', 'Girish@email.com'),
        (3, 'Hari', 'Brindhavan gardens', 'Vijayawada', 'Andhra Pradesh', 'India', '3456789012', 'Hari@email.com'),
        (4, 'Ram', 'Gandhi nagar', 'Chitoor', 'Andhra Pradesh', 'India', '4567890123', 'Ram@email.com'),
        (5, 'Krishna', 'Beach road', 'Nellore', 'Andhra Pradesh', 'India', '5678901234', 'Krishna@email.com');
""")

# Insert data into EMPLOYEES table
cur.execute("""
    INSERT INTO EMPLOYEES (employee_id, employee_name, address, city, state, country, phone_number, email)
    VALUES
        (1, 'Harish', 'Gayatri hospital', 'Kurnool', 'Andhra Pradesh', 'India', '1234567890', 'Harish@email.com'),
        (2, 'Pavan', 'vasavi towers', 'Godavari', 'Andhra Pradesh', 'India', '2345678901', 'Pavan@email.com'),
        (3, 'Kumar', 'Surya road', 'Guntur', 'Andhra Pradesh', 'India', '3456789012', 'Kumar@email.com'),
        (4, 'Abhi', 'balaji nagar', 'Vizag', 'Andhra Pradesh', 'India', '4567890123', 'Abhi@email.com'),
        (5, 'Sai', 'camp road', 'Vizag', 'Andhra Pradesh', 'India', '5678901234', 'Sai@email.com');
""")

# Insert data into WAITERS table
cur.execute("""
    INSERT INTO WAITERS (waiter_id, employee_id, restaurant_id)
    VALUES
        (1, 1, 1),
        (2, 2, 2),
        (3, 3, 3),
        (4, 4, 4),
        (5, 5, 5);
""")

# Insert data into CHEFS table
cur.execute("""
    INSERT INTO CHEFS (chef_id, employee_id, restaurant_id)
    VALUES
        (1, 1, 1),
        (2, 2, 2),
        (3, 3, 3),
        (4, 4, 4),
        (5, 5, 5);
""")

# Insert data into MENUITEMS table
cur.execute("""
    INSERT INTO MENUITEMS (menu_item_id, restaurant_id, item_name, description, price)
    VALUES
        (1, 1, 'Panner butter masala', 'It is made with panner', 10.99),
        (2, 2, 'Kaju capsicum curry', 'It is spicy', 9.99),
        (3, 3, 'Roti', 'Made with wheat', 8.99),
        (4, 4, 'Butter naan', 'Made with maida', 7.99),
        (5, 5, 'Biryani', 'It is spicy', 6.99);
""")

# Insert data into ORDERSTATUSES table
cur.execute("""
    INSERT INTO ORDERSTATUSES (order_status_id, status_name)
    VALUES
        (1, 'In progress'),
        (2, 'Completed'),
        (3, 'In progress'),
        (4, 'Completed'),
        (5, 'In progress');
""")

# Insert data into ORDERS table
cur.execute("""
    INSERT INTO ORDERS (order_id, customer_id, waiter_id, order_status_id, order_date, total_amount)
    VALUES
        (1, 1, 1, 1, '2023-07-01', 50.99),
        (2, 2, 2, 2, '2023-07-02', 40.99),
        (3, 3, 3, 3, '2023-07-03', 30.99),
        (4, 4, 4, 4, '2023-07-04', 20.99),
        (5, 5, 5, 5, '2023-07-05', 10.99);
""")

# Insert data into ORDERLINE table
cur.execute("""
    INSERT INTO ORDERLINE (orderline_id, order_id, menu_item_id, quantity)
    VALUES
        (1, 1, 1, 2),
        (2, 2, 2, 1),
        (3, 3, 3, 3),
        (4, 4, 4, 2),
        (5, 5, 5, 1);
""")

# Insert data into DELIVERYEMPLOYEES table
cur.execute("""
    INSERT INTO DELIVERYEMPLOYEES (delivery_employee_id, employee_id)
    VALUES
        (1, 1),
        (2, 2),
        (3, 3),
        (4, 4),
        (5, 5);
""")

# Insert data into DELIVERY table
cur.execute("""
    INSERT INTO DELIVERY (delivery_id, order_id, delivery_employee_id, delivery_date, address, city, state, country)
    VALUES
        (1, 1, 1, '2023-07-01', 'Ashoke nagar', 'Vizag', 'Andhra Pradesh', 'India'),
        (2, 2, 2, '2023-07-02', 'Atchempet road', 'Guntur', 'Andhra Pradesh', 'India'),
        (3, 3, 3, '2023-07-03', 'Brindhavan gardens', 'Vijayawada', 'Andhra Pradesh', 'India'),
        (4, 4, 4, '2023-07-04', 'Gandhi nagar', 'Chitoor', 'Andhra Pradesh', 'India'),
        (5, 5, 5, '2023-07-05', 'Beach road', 'Nellore', 'Andhra Pradesh', 'India');
""")

# Insert data into DINEINCUSTOMERS table
cur.execute("""
    INSERT INTO DINEINCUSTOMERS (customer_id, table_number)
    VALUES
        (1, 1),
        (2, 2),
        (3, 3),
        (4, 4),
        (5, 5);
""")

# Insert data into DELIVERYCUSTOMERS table
cur.execute("""
    INSERT INTO DELIVERYCUSTOMERS (customer_id, address, city, state, country)
    VALUES
        (1, 'Ashoke nagar', 'Vizag', 'Andhra Pradesh', 'India'),
        (2, 'Atchempet road', 'Guntur', 'Andhra Pradesh', 'India'),
        (3, 'Brindhavan gardens', 'Vijayawada', 'Andhra Pradesh', 'India'),
        (4, 'Gandhi nagar', 'Chitoor', 'Andhra Pradesh', 'India'),
        (5, 'Beach road', 'Nellore', 'Andhra Pradesh', 'India');
""")

# Insert data into FEEDBACK table
cur.execute("""
    INSERT INTO FEEDBACK (feedback_id, customer_id, feedback_date, feedback_text)
    VALUES
        (1, 1, '2023-07-01', 'Good'),
        (2, 2, '2023-07-02', 'Average'),
        (3, 3, '2023-07-03', 'Nice'),
        (4, 4, '2023-07-04', 'Delicious'),
        (5, 5, '2023-07-05', 'Good');
""")

# Commit the changes and close the connection
conn.commit()